# Hive Assignment 2. DML: Find Most Popular Tags

Compare most popular tags in year 2016 with tags in 2009. Tag popularity is the number of questions (post_type_id=1) with this tag. 

Output top 10 tags in format:

```
tag <tab> rank in 2016 <tab> rank in 2009 <tab> tag popularity in 2016 <tab> tag popularity in 2009
```

Example:

```
php 5 3 1234 5678
```

For 'rank' in each year use rank() function. Order by 'rank in 2016'. 

The part of the result on the sample dataset:

```
...
android 3   52  1809    25
php 4   3   1673    215
python  5   11  1585    108
c#  6   1   1519    423
...
```

Please use the tables `posts` and `users` from the database `stackoverflow_`. Database `posts` is partitioned by year and by month.


### Step 1. Find most popular tags

In [1]:
%%writefile query.hql
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE stackoverflow_;

SELECT tag,
       rank() OVER (ORDER BY popularity2016 DESC) as rank2016,
       rank() OVER (ORDER BY popularity2009 DESC) as rank2009,
       popularity2016,
       popularity2009
FROM (
    SELECT tag,
           sum(CASE WHEN year = '2016' THEN 1 ELSE 0 END) as popularity2016,
           sum(CASE WHEN year = '2009' THEN 1 ELSE 0 END) as popularity2009
    FROM posts 
    LATERAL VIEW explode(tags) tagged as tag
    WHERE post_type_id=1 AND year IN ('2009', '2016')
    GROUP BY tag
) as tmp
ORDER BY rank2016
LIMIT 10;

Overwriting query.hql


In [ ]:
%%bash

hive -f query.hql